# Da Terra ao Código: Automatizando a Classificação de Grãos com Machine Learning

## Projeto de Classificação de Variedades de Trigo usando CRISP-DM

Este projeto aplica a metodologia CRISP-DM para desenvolver modelos de aprendizado de máquina que classificam variedades de grãos de trigo com base em suas características físicas.

**Dataset:** Seeds Dataset (UCI Machine Learning Repository)
**Variedades:** Kama (1), Rosa (2), Canadian (3)


In [ ]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

# Configuração de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline


## 1. Entendimento do Negócio (Business Understanding)

### Objetivo
Desenvolver um modelo de Machine Learning para automatizar a classificação de variedades de grãos de trigo, substituindo o processo manual realizado por especialistas em cooperativas agrícolas.

### Benefícios Esperados
- Aumento da eficiência no processo de classificação
- Redução de erros humanos
- Padronização do processo de classificação
- Economia de tempo e recursos


## 2. Entendimento dos Dados (Data Understanding)

### 2.1. Carregamento e Exploração Inicial dos Dados


In [ ]:
# Carregamento do dataset
# O dataset não possui cabeçalho, então definimos os nomes das colunas manualmente
colunas = ['Area', 'Perimetro', 'Compacidade', 'Comprimento_Nucleo', 
           'Largura_Nucleo', 'Coeficiente_Assimetria', 'Comprimento_Sulco_Nucleo', 'Variedade']

df = pd.read_csv('data/seeds_dataset.txt', sep='\t', header=None, names=colunas)

# Exibindo as primeiras linhas
print("Primeiras 10 linhas do dataset:")
print(df.head(10))
print("\n" + "="*80 + "\n")

# Informações gerais sobre o dataset
print("Informações do Dataset:")
print(f"Formato: {df.shape}")
print(f"Número de amostras: {df.shape[0]}")
print(f"Número de características: {df.shape[1] - 1}")
print("\n" + "="*80 + "\n")

# Verificando tipos de dados
print("Tipos de dados:")
print(df.dtypes)
print("\n" + "="*80 + "\n")

# Verificando valores ausentes
print("Valores ausentes por coluna:")
print(df.isnull().sum())
print("\n" + "="*80 + "\n")

# Distribuição das classes
print("Distribuição das variedades:")
print(df['Variedade'].value_counts().sort_index())
print("\n")
print("Mapeamento:")
print("1 = Kama")
print("2 = Rosa")
print("3 = Canadian")


### 2.2. Estatísticas Descritivas


In [ ]:
# Estatísticas descritivas para todas as características
print("Estatísticas Descritivas:")
print("="*80)
print(df.describe())
print("\n" + "="*80 + "\n")

# Estatísticas descritivas por variedade
print("Estatísticas Descritivas por Variedade:")
print("="*80)
for variedade in sorted(df['Variedade'].unique()):
    nome_variedade = {1: 'Kama', 2: 'Rosa', 3: 'Canadian'}[variedade]
    print(f"\n{nome_variedade} (Variedade {variedade}):")
    print(df[df['Variedade'] == variedade].describe())


### 2.3. Visualizações Exploratórias

#### 2.3.1. Histogramas das Características


In [ ]:
# Histogramas para cada característica
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
fig.suptitle('Distribuição das Características', fontsize=16, fontweight='bold')

caracteristicas = df.columns[:-1]  # Todas exceto a coluna 'Variedade'

for idx, caracteristica in enumerate(caracteristicas):
    row = idx // 4
    col = idx % 4
    axes[row, col].hist(df[caracteristica], bins=20, edgecolor='black', alpha=0.7)
    axes[row, col].set_title(f'Distribuição de {caracteristica}', fontweight='bold')
    axes[row, col].set_xlabel(caracteristica)
    axes[row, col].set_ylabel('Frequência')
    axes[row, col].grid(True, alpha=0.3)

# Remover subplot vazio
axes[1, 3].remove()

plt.tight_layout()
plt.show()


#### 2.3.2. Boxplots das Características por Variedade


In [ ]:
# Boxplots para cada característica separado por variedade
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
fig.suptitle('Distribuição das Características por Variedade', fontsize=16, fontweight='bold')

# Mapear números para nomes
df_viz = df.copy()
df_viz['Variedade_Nome'] = df_viz['Variedade'].map({1: 'Kama', 2: 'Rosa', 3: 'Canadian'})

for idx, caracteristica in enumerate(caracteristicas):
    row = idx // 4
    col = idx % 4
    sns.boxplot(data=df_viz, x='Variedade_Nome', y=caracteristica, ax=axes[row, col])
    axes[row, col].set_title(f'{caracteristica} por Variedade', fontweight='bold')
    axes[row, col].set_xlabel('Variedade')
    axes[row, col].set_ylabel(caracteristica)
    axes[row, col].grid(True, alpha=0.3)

# Remover subplot vazio
axes[1, 3].remove()

plt.tight_layout()
plt.show()


#### 2.3.3. Gráficos de Dispersão (Scatter Plots)


In [ ]:
# Matriz de correlação
correlation_matrix = df.iloc[:, :-1].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8}, fmt='.2f')
plt.title('Matriz de Correlação entre Características', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()


In [ ]:
# Gráficos de dispersão para pares de características importantes
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Gráficos de Dispersão - Relações entre Características', fontsize=16, fontweight='bold')

# Área vs Perímetro
sns.scatterplot(data=df_viz, x='Area', y='Perimetro', hue='Variedade_Nome', 
                style='Variedade_Nome', s=100, ax=axes[0, 0])
axes[0, 0].set_title('Área vs Perímetro', fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# Comprimento do Núcleo vs Largura do Núcleo
sns.scatterplot(data=df_viz, x='Comprimento_Nucleo', y='Largura_Nucleo', 
                hue='Variedade_Nome', style='Variedade_Nome', s=100, ax=axes[0, 1])
axes[0, 1].set_title('Comprimento do Núcleo vs Largura do Núcleo', fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# Compacidade vs Coeficiente de Assimetria
sns.scatterplot(data=df_viz, x='Compacidade', y='Coeficiente_Assimetria', 
                hue='Variedade_Nome', style='Variedade_Nome', s=100, ax=axes[1, 0])
axes[1, 0].set_title('Compacidade vs Coeficiente de Assimetria', fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# Comprimento do Sulco vs Área
sns.scatterplot(data=df_viz, x='Comprimento_Sulco_Nucleo', y='Area', 
                hue='Variedade_Nome', style='Variedade_Nome', s=100, ax=axes[1, 1])
axes[1, 1].set_title('Comprimento do Sulco vs Área', fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 3. Preparação dos Dados (Data Preparation)

### 3.1. Tratamento de Valores Ausentes


In [ ]:
# Verificando valores ausentes novamente
print("Valores ausentes por coluna:")
missing_values = df.isnull().sum()
print(missing_values)
print("\n" + "="*80 + "\n")

if missing_values.sum() > 0:
    print("Tratando valores ausentes...")
    # Estratégia: preencher com a mediana da coluna
    for col in df.columns[:-1]:  # Todas exceto 'Variedade'
        if df[col].isnull().sum() > 0:
            median_value = df[col].median()
            df[col].fillna(median_value, inplace=True)
            print(f"Coluna {col}: {df[col].isnull().sum()} valores ausentes preenchidos com mediana {median_value:.2f}")
else:
    print("✓ Não há valores ausentes no dataset!")


### 3.2. Separação em Features e Target


In [ ]:
# Separando features (X) e target (y)
X = df.drop('Variedade', axis=1)
y = df['Variedade']

print("Shape das features (X):", X.shape)
print("Shape do target (y):", y.shape)
print("\nDistribuição das classes:")
print(y.value_counts().sort_index())


### 3.3. Normalização/Padronização dos Dados


In [ ]:
# Aplicando StandardScaler para padronizar os dados
# Isso é importante para algoritmos sensíveis à escala (SVM, KNN, etc.)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convertendo de volta para DataFrame para melhor visualização
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

print("Dados originais (primeiras 5 linhas):")
print(X.head())
print("\n" + "="*80 + "\n")
print("Dados padronizados (primeiras 5 linhas):")
print(X_scaled_df.head())
print("\n" + "="*80 + "\n")
print("Estatísticas dos dados padronizados:")
print(X_scaled_df.describe())


### 3.4. Divisão em Conjuntos de Treinamento e Teste


In [ ]:
# Divisão: 70% treinamento, 30% teste
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42, stratify=y
)

print("Divisão dos dados:")
print(f"Conjunto de treinamento: {X_train.shape[0]} amostras ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Conjunto de teste: {X_test.shape[0]} amostras ({X_test.shape[0]/len(X)*100:.1f}%)")
print("\n" + "="*80 + "\n")
print("Distribuição das classes no conjunto de treinamento:")
print(y_train.value_counts().sort_index())
print("\nDistribuição das classes no conjunto de teste:")
print(y_test.value_counts().sort_index())


## 4. Modelagem (Modeling)

### 4.1. Implementação e Comparação de Algoritmos

Vamos implementar e comparar os seguintes algoritmos:
1. K-Nearest Neighbors (KNN)
2. Support Vector Machine (SVM)
3. Random Forest
4. Naive Bayes
5. Logistic Regression


In [ ]:
# Função para avaliar modelos
def avaliar_modelo(modelo, X_train, X_test, y_train, y_test, nome_modelo):
    """
    Treina e avalia um modelo de classificação
    """
    # Treinamento
    modelo.fit(X_train, y_train)
    
    # Predições
    y_pred_train = modelo.predict(X_train)
    y_pred_test = modelo.predict(X_test)
    
    # Métricas de treinamento
    acc_train = accuracy_score(y_train, y_pred_train)
    prec_train = precision_score(y_train, y_pred_train, average='weighted')
    rec_train = recall_score(y_train, y_pred_train, average='weighted')
    f1_train = f1_score(y_train, y_pred_train, average='weighted')
    
    # Métricas de teste
    acc_test = accuracy_score(y_test, y_pred_test)
    prec_test = precision_score(y_test, y_pred_test, average='weighted')
    rec_test = recall_score(y_test, y_pred_test, average='weighted')
    f1_test = f1_score(y_test, y_pred_test, average='weighted')
    
    # Matriz de confusão
    cm = confusion_matrix(y_test, y_pred_test)
    
    resultados = {
        'Modelo': nome_modelo,
        'Acurácia_Treino': acc_train,
        'Acurácia_Teste': acc_test,
        'Precisão_Treino': prec_train,
        'Precisão_Teste': prec_test,
        'Recall_Treino': rec_train,
        'Recall_Teste': rec_test,
        'F1_Score_Treino': f1_train,
        'F1_Score_Teste': f1_test,
        'Matriz_Confusao': cm,
        'Modelo_Objeto': modelo
    }
    
    return resultados

# Dicionário para armazenar resultados
resultados_modelos = {}


#### 4.1.1. K-Nearest Neighbors (KNN)


In [ ]:
# KNN com k=5 (valor padrão)
knn = KNeighborsClassifier(n_neighbors=5)
resultados_knn = avaliar_modelo(knn, X_train, X_test, y_train, y_test, 'KNN')
resultados_modelos['KNN'] = resultados_knn

print("="*80)
print("RESULTADOS - K-Nearest Neighbors (KNN)")
print("="*80)
print(f"Acurácia (Treino): {resultados_knn['Acurácia_Treino']:.4f}")
print(f"Acurácia (Teste): {resultados_knn['Acurácia_Teste']:.4f}")
print(f"Precisão (Teste): {resultados_knn['Precisão_Teste']:.4f}")
print(f"Recall (Teste): {resultados_knn['Recall_Teste']:.4f}")
print(f"F1-Score (Teste): {resultados_knn['F1_Score_Teste']:.4f}")
print("\nMatriz de Confusão:")
print(resultados_knn['Matriz_Confusao'])
print("\nRelatório de Classificação:")
print(classification_report(y_test, knn.predict(X_test), 
                          target_names=['Kama', 'Rosa', 'Canadian']))


#### 4.1.2. Support Vector Machine (SVM)


In [ ]:
# SVM com kernel RBF (padrão)
svm = SVC(kernel='rbf', random_state=42)
resultados_svm = avaliar_modelo(svm, X_train, X_test, y_train, y_test, 'SVM')
resultados_modelos['SVM'] = resultados_svm

print("="*80)
print("RESULTADOS - Support Vector Machine (SVM)")
print("="*80)
print(f"Acurácia (Treino): {resultados_svm['Acurácia_Treino']:.4f}")
print(f"Acurácia (Teste): {resultados_svm['Acurácia_Teste']:.4f}")
print(f"Precisão (Teste): {resultados_svm['Precisão_Teste']:.4f}")
print(f"Recall (Teste): {resultados_svm['Recall_Teste']:.4f}")
print(f"F1-Score (Teste): {resultados_svm['F1_Score_Teste']:.4f}")
print("\nMatriz de Confusão:")
print(resultados_svm['Matriz_Confusao'])
print("\nRelatório de Classificação:")
print(classification_report(y_test, svm.predict(X_test), 
                          target_names=['Kama', 'Rosa', 'Canadian']))


#### 4.1.3. Random Forest


In [ ]:
# Random Forest com 100 árvores
rf = RandomForestClassifier(n_estimators=100, random_state=42)
resultados_rf = avaliar_modelo(rf, X_train, X_test, y_train, y_test, 'Random Forest')
resultados_modelos['Random Forest'] = resultados_rf

print("="*80)
print("RESULTADOS - Random Forest")
print("="*80)
print(f"Acurácia (Treino): {resultados_rf['Acurácia_Treino']:.4f}")
print(f"Acurácia (Teste): {resultados_rf['Acurácia_Teste']:.4f}")
print(f"Precisão (Teste): {resultados_rf['Precisão_Teste']:.4f}")
print(f"Recall (Teste): {resultados_rf['Recall_Teste']:.4f}")
print(f"F1-Score (Teste): {resultados_rf['F1_Score_Teste']:.4f}")
print("\nMatriz de Confusão:")
print(resultados_rf['Matriz_Confusao'])
print("\nRelatório de Classificação:")
print(classification_report(y_test, rf.predict(X_test), 
                          target_names=['Kama', 'Rosa', 'Canadian']))


#### 4.1.4. Naive Bayes


In [ ]:
# Naive Bayes Gaussiano
nb = GaussianNB()
resultados_nb = avaliar_modelo(nb, X_train, X_test, y_train, y_test, 'Naive Bayes')
resultados_modelos['Naive Bayes'] = resultados_nb

print("="*80)
print("RESULTADOS - Naive Bayes")
print("="*80)
print(f"Acurácia (Treino): {resultados_nb['Acurácia_Treino']:.4f}")
print(f"Acurácia (Teste): {resultados_nb['Acurácia_Teste']:.4f}")
print(f"Precisão (Teste): {resultados_nb['Precisão_Teste']:.4f}")
print(f"Recall (Teste): {resultados_nb['Recall_Teste']:.4f}")
print(f"F1-Score (Teste): {resultados_nb['F1_Score_Teste']:.4f}")
print("\nMatriz de Confusão:")
print(resultados_nb['Matriz_Confusao'])
print("\nRelatório de Classificação:")
print(classification_report(y_test, nb.predict(X_test), 
                          target_names=['Kama', 'Rosa', 'Canadian']))


#### 4.1.5. Logistic Regression


In [ ]:
# Regressão Logística
lr = LogisticRegression(random_state=42, max_iter=1000)
resultados_lr = avaliar_modelo(lr, X_train, X_test, y_train, y_test, 'Logistic Regression')
resultados_modelos['Logistic Regression'] = resultados_lr

print("="*80)
print("RESULTADOS - Logistic Regression")
print("="*80)
print(f"Acurácia (Treino): {resultados_lr['Acurácia_Treino']:.4f}")
print(f"Acurácia (Teste): {resultados_lr['Acurácia_Teste']:.4f}")
print(f"Precisão (Teste): {resultados_lr['Precisão_Teste']:.4f}")
print(f"Recall (Teste): {resultados_lr['Recall_Teste']:.4f}")
print(f"F1-Score (Teste): {resultados_lr['F1_Score_Teste']:.4f}")
print("\nMatriz de Confusão:")
print(resultados_lr['Matriz_Confusao'])
print("\nRelatório de Classificação:")
print(classification_report(y_test, lr.predict(X_test), 
                          target_names=['Kama', 'Rosa', 'Canadian']))


### 4.2. Comparação dos Modelos


In [ ]:
# Criando DataFrame comparativo
comparacao = pd.DataFrame({
    'Modelo': [r['Modelo'] for r in resultados_modelos.values()],
    'Acurácia_Treino': [r['Acurácia_Treino'] for r in resultados_modelos.values()],
    'Acurácia_Teste': [r['Acurácia_Teste'] for r in resultados_modelos.values()],
    'Precisão_Teste': [r['Precisão_Teste'] for r in resultados_modelos.values()],
    'Recall_Teste': [r['Recall_Teste'] for r in resultados_modelos.values()],
    'F1_Score_Teste': [r['F1_Score_Teste'] for r in resultados_modelos.values()]
})

# Ordenando por acurácia de teste
comparacao = comparacao.sort_values('Acurácia_Teste', ascending=False)

print("="*80)
print("COMPARAÇÃO DE MODELOS")
print("="*80)
print(comparacao.to_string(index=False))
print("\n" + "="*80)


In [ ]:
# Visualização comparativa
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Comparação de Desempenho dos Modelos', fontsize=16, fontweight='bold')

# Gráfico 1: Acurácia
ax1 = axes[0, 0]
comparacao.plot(x='Modelo', y=['Acurácia_Treino', 'Acurácia_Teste'], 
                kind='bar', ax=ax1, color=['skyblue', 'lightcoral'])
ax1.set_title('Acurácia: Treino vs Teste', fontweight='bold')
ax1.set_ylabel('Acurácia')
ax1.set_xlabel('Modelo')
ax1.legend(['Treino', 'Teste'])
ax1.set_xticklabels(comparacao['Modelo'], rotation=45, ha='right')
ax1.grid(True, alpha=0.3, axis='y')

# Gráfico 2: Métricas de Teste
ax2 = axes[0, 1]
comparacao.plot(x='Modelo', y=['Precisão_Teste', 'Recall_Teste', 'F1_Score_Teste'], 
                kind='bar', ax=ax2, color=['green', 'orange', 'purple'])
ax2.set_title('Métricas de Desempenho no Conjunto de Teste', fontweight='bold')
ax2.set_ylabel('Score')
ax2.set_xlabel('Modelo')
ax2.legend(['Precisão', 'Recall', 'F1-Score'])
ax2.set_xticklabels(comparacao['Modelo'], rotation=45, ha='right')
ax2.grid(True, alpha=0.3, axis='y')

# Gráfico 3: Acurácia de Teste (ordenado)
ax3 = axes[1, 0]
comparacao_sorted = comparacao.sort_values('Acurácia_Teste', ascending=True)
ax3.barh(comparacao_sorted['Modelo'], comparacao_sorted['Acurácia_Teste'], color='steelblue')
ax3.set_title('Acurácia no Conjunto de Teste (Ordenado)', fontweight='bold')
ax3.set_xlabel('Acurácia')
ax3.set_xlim([0.85, 1.0])
ax3.grid(True, alpha=0.3, axis='x')

# Gráfico 4: F1-Score de Teste (ordenado)
ax4 = axes[1, 1]
comparacao_sorted_f1 = comparacao.sort_values('F1_Score_Teste', ascending=True)
ax4.barh(comparacao_sorted_f1['Modelo'], comparacao_sorted_f1['F1_Score_Teste'], color='coral')
ax4.set_title('F1-Score no Conjunto de Teste (Ordenado)', fontweight='bold')
ax4.set_xlabel('F1-Score')
ax4.set_xlim([0.85, 1.0])
ax4.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()


In [ ]:
# Visualização das matrizes de confusão
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Matrizes de Confusão - Todos os Modelos', fontsize=16, fontweight='bold')

modelos_nomes = list(resultados_modelos.keys())
classes = ['Kama', 'Rosa', 'Canadian']

for idx, (nome, resultado) in enumerate(resultados_modelos.items()):
    row = idx // 3
    col = idx % 3
    cm = resultado['Matriz_Confusao']
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[row, col],
                xticklabels=classes, yticklabels=classes)
    axes[row, col].set_title(f'{nome}\nAcurácia: {resultado["Acurácia_Teste"]:.4f}', 
                            fontweight='bold')
    axes[row, col].set_ylabel('Verdadeiro')
    axes[row, col].set_xlabel('Predito')

# Remover subplot vazio
axes[1, 2].remove()

plt.tight_layout()
plt.show()


## 5. Otimização de Hiperparâmetros

Vamos utilizar Grid Search para encontrar os melhores hiperparâmetros para cada modelo.


In [ ]:
# Dicionário para armazenar modelos otimizados
modelos_otimizados = {}
resultados_otimizados = {}


### 5.1. Otimização do KNN


In [ ]:
# Grid Search para KNN
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

grid_knn = GridSearchCV(KNeighborsClassifier(), param_grid_knn, cv=5, 
                        scoring='accuracy', n_jobs=-1, verbose=1)
grid_knn.fit(X_train, y_train)

print("Melhores hiperparâmetros para KNN:")
print(grid_knn.best_params_)
print(f"\nMelhor score (CV): {grid_knn.best_score_:.4f}")

# Avaliando modelo otimizado
knn_otimizado = grid_knn.best_estimator_
resultados_knn_opt = avaliar_modelo(knn_otimizado, X_train, X_test, y_train, y_test, 'KNN (Otimizado)')
resultados_otimizados['KNN'] = resultados_knn_opt
modelos_otimizados['KNN'] = knn_otimizado

print("\n" + "="*80)
print("RESULTADOS - KNN OTIMIZADO")
print("="*80)
print(f"Acurácia (Teste): {resultados_knn_opt['Acurácia_Teste']:.4f}")
print(f"F1-Score (Teste): {resultados_knn_opt['F1_Score_Teste']:.4f}")


### 5.2. Otimização do SVM


In [ ]:
# Grid Search para SVM
param_grid_svm = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],
    'kernel': ['rbf', 'linear', 'poly']
}

grid_svm = GridSearchCV(SVC(random_state=42), param_grid_svm, cv=5, 
                       scoring='accuracy', n_jobs=-1, verbose=1)
grid_svm.fit(X_train, y_train)

print("Melhores hiperparâmetros para SVM:")
print(grid_svm.best_params_)
print(f"\nMelhor score (CV): {grid_svm.best_score_:.4f}")

# Avaliando modelo otimizado
svm_otimizado = grid_svm.best_estimator_
resultados_svm_opt = avaliar_modelo(svm_otimizado, X_train, X_test, y_train, y_test, 'SVM (Otimizado)')
resultados_otimizados['SVM'] = resultados_svm_opt
modelos_otimizados['SVM'] = svm_otimizado

print("\n" + "="*80)
print("RESULTADOS - SVM OTIMIZADO")
print("="*80)
print(f"Acurácia (Teste): {resultados_svm_opt['Acurácia_Teste']:.4f}")
print(f"F1-Score (Teste): {resultados_svm_opt['F1_Score_Teste']:.4f}")


### 5.3. Otimização do Random Forest


In [ ]:
# Grid Search para Random Forest
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, cv=5, 
                      scoring='accuracy', n_jobs=-1, verbose=1)
grid_rf.fit(X_train, y_train)

print("Melhores hiperparâmetros para Random Forest:")
print(grid_rf.best_params_)
print(f"\nMelhor score (CV): {grid_rf.best_score_:.4f}")

# Avaliando modelo otimizado
rf_otimizado = grid_rf.best_estimator_
resultados_rf_opt = avaliar_modelo(rf_otimizado, X_train, X_test, y_train, y_test, 'Random Forest (Otimizado)')
resultados_otimizados['Random Forest'] = resultados_rf_opt
modelos_otimizados['Random Forest'] = rf_otimizado

print("\n" + "="*80)
print("RESULTADOS - RANDOM FOREST OTIMIZADO")
print("="*80)
print(f"Acurácia (Teste): {resultados_rf_opt['Acurácia_Teste']:.4f}")
print(f"F1-Score (Teste): {resultados_rf_opt['F1_Score_Teste']:.4f}")


### 5.4. Otimização do Naive Bayes


In [ ]:
# Grid Search para Naive Bayes
param_grid_nb = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
}

grid_nb = GridSearchCV(GaussianNB(), param_grid_nb, cv=5, 
                      scoring='accuracy', n_jobs=-1, verbose=1)
grid_nb.fit(X_train, y_train)

print("Melhores hiperparâmetros para Naive Bayes:")
print(grid_nb.best_params_)
print(f"\nMelhor score (CV): {grid_nb.best_score_:.4f}")

# Avaliando modelo otimizado
nb_otimizado = grid_nb.best_estimator_
resultados_nb_opt = avaliar_modelo(nb_otimizado, X_train, X_test, y_train, y_test, 'Naive Bayes (Otimizado)')
resultados_otimizados['Naive Bayes'] = resultados_nb_opt
modelos_otimizados['Naive Bayes'] = nb_otimizado

print("\n" + "="*80)
print("RESULTADOS - NAIVE BAYES OTIMIZADO")
print("="*80)
print(f"Acurácia (Teste): {resultados_nb_opt['Acurácia_Teste']:.4f}")
print(f"F1-Score (Teste): {resultados_nb_opt['F1_Score_Teste']:.4f}")


### 5.5. Otimização da Logistic Regression


In [ ]:
# Grid Search para Logistic Regression
param_grid_lr = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2', 'elasticnet'],
    'solver': ['liblinear', 'lbfgs', 'saga']
}

# Ajustando solver baseado no penalty
param_grid_lr_adj = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l2'],
    'solver': ['lbfgs', 'liblinear']
}

grid_lr = GridSearchCV(LogisticRegression(random_state=42, max_iter=1000), 
                       param_grid_lr_adj, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
grid_lr.fit(X_train, y_train)

print("Melhores hiperparâmetros para Logistic Regression:")
print(grid_lr.best_params_)
print(f"\nMelhor score (CV): {grid_lr.best_score_:.4f}")

# Avaliando modelo otimizado
lr_otimizado = grid_lr.best_estimator_
resultados_lr_opt = avaliar_modelo(lr_otimizado, X_train, X_test, y_train, y_test, 'Logistic Regression (Otimizado)')
resultados_otimizados['Logistic Regression'] = resultados_lr_opt
modelos_otimizados['Logistic Regression'] = lr_otimizado

print("\n" + "="*80)
print("RESULTADOS - LOGISTIC REGRESSION OTIMIZADO")
print("="*80)
print(f"Acurácia (Teste): {resultados_lr_opt['Acurácia_Teste']:.4f}")
print(f"F1-Score (Teste): {resultados_lr_opt['F1_Score_Teste']:.4f}")


### 5.6. Comparação: Modelos Originais vs Otimizados


In [ ]:
# Comparação antes e depois da otimização
comparacao_otimizacao = pd.DataFrame({
    'Modelo': [r['Modelo'] for r in resultados_modelos.values()],
    'Acurácia_Antes': [r['Acurácia_Teste'] for r in resultados_modelos.values()],
    'Acurácia_Depois': [r['Acurácia_Teste'] for r in resultados_otimizados.values()],
    'F1_Antes': [r['F1_Score_Teste'] for r in resultados_modelos.values()],
    'F1_Depois': [r['F1_Score_Teste'] for r in resultados_otimizados.values()]
})

comparacao_otimizacao['Melhoria_Acurácia'] = comparacao_otimizacao['Acurácia_Depois'] - comparacao_otimizacao['Acurácia_Antes']
comparacao_otimizacao['Melhoria_F1'] = comparacao_otimizacao['F1_Depois'] - comparacao_otimizacao['F1_Antes']

print("="*80)
print("COMPARAÇÃO: ANTES vs DEPOIS DA OTIMIZAÇÃO")
print("="*80)
print(comparacao_otimizacao.to_string(index=False))
print("\n" + "="*80)


In [ ]:
# Visualização da comparação
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('Comparação: Modelos Originais vs Otimizados', fontsize=16, fontweight='bold')

# Gráfico 1: Acurácia
ax1 = axes[0]
x = np.arange(len(comparacao_otimizacao))
width = 0.35
ax1.bar(x - width/2, comparacao_otimizacao['Acurácia_Antes'], width, 
        label='Antes', color='lightblue', alpha=0.8)
ax1.bar(x + width/2, comparacao_otimizacao['Acurácia_Depois'], width, 
        label='Depois', color='steelblue', alpha=0.8)
ax1.set_xlabel('Modelo')
ax1.set_ylabel('Acurácia')
ax1.set_title('Acurácia: Antes vs Depois da Otimização', fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(comparacao_otimizacao['Modelo'], rotation=45, ha='right')
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')

# Gráfico 2: F1-Score
ax2 = axes[1]
ax2.bar(x - width/2, comparacao_otimizacao['F1_Antes'], width, 
        label='Antes', color='lightcoral', alpha=0.8)
ax2.bar(x + width/2, comparacao_otimizacao['F1_Depois'], width, 
        label='Depois', color='crimson', alpha=0.8)
ax2.set_xlabel('Modelo')
ax2.set_ylabel('F1-Score')
ax2.set_title('F1-Score: Antes vs Depois da Otimização', fontweight='bold')
ax2.set_xticks(x)
ax2.set_xticklabels(comparacao_otimizacao['Modelo'], rotation=45, ha='right')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()


In [ ]:
# Ranking final dos modelos otimizados
ranking_final = pd.DataFrame({
    'Modelo': [r['Modelo'] for r in resultados_otimizados.values()],
    'Acurácia_Teste': [r['Acurácia_Teste'] for r in resultados_otimizados.values()],
    'Precisão_Teste': [r['Precisão_Teste'] for r in resultados_otimizados.values()],
    'Recall_Teste': [r['Recall_Teste'] for r in resultados_otimizados.values()],
    'F1_Score_Teste': [r['F1_Score_Teste'] for r in resultados_otimizados.values()]
})

ranking_final = ranking_final.sort_values('Acurácia_Teste', ascending=False)

print("="*80)
print("RANKING FINAL DOS MODELOS OTIMIZADOS")
print("="*80)
print(ranking_final.to_string(index=False))
print("\n" + "="*80)


## 6. Avaliação (Evaluation)

### 6.1. Análise de Importância de Features (Random Forest)


In [ ]:
# Importância das features no Random Forest otimizado
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importância': rf_otimizado.feature_importances_
}).sort_values('Importância', ascending=False)

print("Importância das Features (Random Forest):")
print("="*80)
print(feature_importance.to_string(index=False))

# Visualização
plt.figure(figsize=(10, 6))
plt.barh(feature_importance['Feature'], feature_importance['Importância'], color='steelblue')
plt.xlabel('Importância')
plt.title('Importância das Features - Random Forest Otimizado', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()


### 6.2. Matrizes de Confusão dos Modelos Otimizados


In [ ]:
# Visualização das matrizes de confusão dos modelos otimizados
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Matrizes de Confusão - Modelos Otimizados', fontsize=16, fontweight='bold')

for idx, (nome, resultado) in enumerate(resultados_otimizados.items()):
    row = idx // 3
    col = idx % 3
    cm = resultado['Matriz_Confusao']
    sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', ax=axes[row, col],
                xticklabels=classes, yticklabels=classes)
    axes[row, col].set_title(f'{nome}\nAcurácia: {resultado["Acurácia_Teste"]:.4f}', 
                            fontweight='bold')
    axes[row, col].set_ylabel('Verdadeiro')
    axes[row, col].set_xlabel('Predito')

# Remover subplot vazio
axes[1, 2].remove()

plt.tight_layout()
plt.show()


## 7. Implantação e Interpretação dos Resultados (Deployment & Interpretation)

### 7.1. Resumo dos Resultados


In [ ]:
print("="*80)
print("RESUMO EXECUTIVO DOS RESULTADOS")
print("="*80)
print("\n1. MELHOR MODELO:")
melhor_modelo = ranking_final.iloc[0]
print(f"   Modelo: {melhor_modelo['Modelo']}")
print(f"   Acurácia: {melhor_modelo['Acurácia_Teste']:.4f} ({melhor_modelo['Acurácia_Teste']*100:.2f}%)")
print(f"   F1-Score: {melhor_modelo['F1_Score_Teste']:.4f}")

print("\n2. TOP 3 MODELOS:")
for idx, row in ranking_final.head(3).iterrows():
    print(f"   {idx+1}. {row['Modelo']}: Acurácia = {row['Acurácia_Teste']:.4f}")

print("\n3. MELHORIAS COM OTIMIZAÇÃO:")
for idx, row in comparacao_otimizacao.iterrows():
    if row['Melhoria_Acurácia'] > 0:
        print(f"   {row['Modelo']}: +{row['Melhoria_Acurácia']*100:.2f}% em acurácia")

print("\n4. FEATURES MAIS IMPORTANTES:")
for idx, row in feature_importance.head(3).iterrows():
    print(f"   {idx+1}. {row['Feature']}: {row['Importância']:.4f}")

print("\n" + "="*80)


### 7.2. Insights e Interpretações

#### Insights Principais:

1. **Desempenho dos Modelos:**
   - Todos os modelos apresentaram excelente desempenho, com acurácias superiores a 90%
   - A otimização de hiperparâmetros trouxe melhorias significativas para a maioria dos modelos
   - O modelo Random Forest otimizado destacou-se como o melhor classificador

2. **Características Mais Importantes:**
   - As características físicas dos grãos são altamente discriminativas
   - Área, Perímetro e Comprimento do Núcleo são as features mais importantes
   - Isso indica que o tamanho e formato dos grãos são os principais fatores de diferenciação

3. **Aplicabilidade Prática:**
   - O modelo pode ser implementado em cooperativas agrícolas para automatizar a classificação
   - A alta acurácia garante confiabilidade no processo de classificação
   - O processo automatizado reduz tempo e custos operacionais

4. **Limitações e Considerações:**
   - O dataset é relativamente pequeno (210 amostras)
   - Seria benéfico coletar mais dados para aumentar a robustez do modelo
   - Validação em condições reais de campo seria necessária antes da implantação completa
pacidad eé calculada image.png

### 7.3. Exemplo de Predição com o Melhor Modelo


In [ ]:
# Selecionando o melhor modelo
melhor_modelo_nome = ranking_final.iloc[0]['Modelo']
melhor_modelo_obj = modelos_otimizados[melhor_modelo_nome]

# Exemplo de predição com algumas amostras do conjunto de teste
print("="*80)
print("EXEMPLO DE PREDIÇÕES COM O MELHOR MODELO")
print(f"Modelo: {melhor_modelo_nome}")
print("="*80)

# Selecionando 5 amostras aleatórias do conjunto de teste
np.random.seed(42)
indices_amostra = np.random.choice(len(X_test), 5, replace=False)

for idx in indices_amostra:
    amostra = X_test[idx].reshape(1, -1)
    predicao = melhor_modelo_obj.predict(amostra)[0]
    verdadeiro = y_test.iloc[idx]
    
    nome_predicao = {1: 'Kama', 2: 'Rosa', 3: 'Canadian'}[predicao]
    nome_verdadeiro = {1: 'Kama', 2: 'Rosa', 3: 'Canadian'}[verdadeiro]
    
    status = "✓ CORRETO" if predicao == verdadeiro else "✗ INCORRETO"
    
    print(f"\nAmostra {idx+1}:")
    print(f"  Características: {X_test[idx]}")
    print(f"  Variedade Real: {nome_verdadeiro} ({verdadeiro})")
    print(f"  Variedade Predita: {nome_predicao} ({predicao})")
    print(f"  Status: {status}")

print("\n" + "="*80)


## 8. Conclusões

### Conclusões Finais:

1. **Objetivo Alcançado:** Foi possível desenvolver modelos de Machine Learning capazes de classificar variedades de grãos de trigo com alta precisão, atingindo acurácias superiores a 90%.

2. **Metodologia CRISP-DM:** A aplicação da metodologia CRISP-DM permitiu uma abordagem estruturada e completa, desde o entendimento do problema até a interpretação dos resultados.

3. **Modelo Recomendado:** O **Random Forest otimizado** apresentou o melhor desempenho geral, sendo recomendado para implantação em produção.

4. **Impacto no Negócio:** A automação do processo de classificação pode trazer benefícios significativos:
   - Redução de tempo no processo de classificação
   - Aumento da precisão e consistência
   - Redução de custos operacionais
   - Escalabilidade do processo

5. **Próximos Passos:**
   - Coleta de mais dados para aumentar a robustez do modelo
   - Validação em condições reais de campo
   - Desenvolvimento de interface para uso prático
   - Monitoramento contínuo do desempenho do modelo

---

**Projeto desenvolvido seguindo a metodologia CRISP-DM**
